# Лабораторна: Реалізація та Аналіз Дерев Рішень для Класифікації

**Мета:** реалізувати дерево рішень з Gini, compute feature importance, implement pruning (cost-complexity), compare with sklearn DecisionTree & RandomForest, perform feature engineering and feature selection.

_Всі кроки виконані у коді нижче — просто запустіть всі клітинки в Jupyter / Colab._

In [ ]:
# Блок 0 — Імпорти та налаштування
import numpy as np, pandas as pd, matplotlib.pyplot as plt, copy, os, time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, matthews_corrcoef, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
%matplotlib inline
print("Ready.")

In [ ]:
# Блок 1 — Створення (або завантаження) датасету
data_path = 'synthetic_coffee_health_10000.csv'
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print("Loaded dataset from", data_path, "shape:", df.shape)
else:
    print("Dataset not found. Generating synthetic dataset (10,000 samples) for demonstration.")
    np.random.seed(42)
    N = 10000
    ids = np.arange(1, N+1)
    age = np.random.randint(18, 80, size=N)
    gender = np.random.choice(['Male','Female','Other'], size=N, p=[0.48,0.48,0.04])
    countries = np.random.choice(['USA','UK','France','Germany','India','Brazil','Kenya'], size=N)
    occupation = np.random.choice(['Student','Employed','Healthcare','Retired','Unemployed','Other'], size=N)
    coffee_intake = np.clip(np.round(np.random.gamma(2.0,1.2,size=N)), 0, 15)
    caffeine_mg = coffee_intake * np.random.normal(95, 10, size=N)
    sleep_hours = np.clip(np.random.normal(7, 1.5, size=N), 3, 10)
    sleep_quality = np.random.choice(['Poor','Average','Good'], size=N, p=[0.2,0.5,0.3])
    stress_level = np.random.choice(['Low','Medium','High'], size=N, p=[0.4,0.4,0.2])
    smoking = np.random.choice(['Never','Former','Current'], size=N, p=[0.7,0.15,0.15])
    alcohol = np.random.choice(['None','Low','Moderate','High'], size=N, p=[0.2,0.5,0.25,0.05])
    bmi = np.round(np.random.normal(25,4,size=N),1)
    heart_rate = np.round(np.random.normal(72,10,size=N)).astype(int)
    physical_activity = np.clip(np.round(np.random.exponential(1.5,size=N),1), 0, 10)
    score = (coffee_intake/5.0) - (sleep_hours/8.0) + (np.isin(stress_level, ['High']).astype(int)*0.8) +             (np.isin(smoking, ['Current']).astype(int)*0.6) + (np.isin(alcohol, ['High']).astype(int)*0.6) +             ((bmi-24)/10.0) - (physical_activity/5.0) + np.random.normal(0,0.5,size=N)
    q = np.quantile(score, [0.25,0.5,0.75])
    labels = np.where(score <= q[0], 'None', np.where(score <= q[1], 'Mild', np.where(score <= q[2], 'Moderate', 'Severe')))
    df = pd.DataFrame({'ID': ids,'Age': age,'Gender': gender,'Country': countries,'Occupation': occupation,'Coffee_Intake': coffee_intake,'Caffeine_mg': np.round(caffeine_mg,1),'Sleep_Hours': np.round(sleep_hours,2),'Sleep_Quality': sleep_quality,'Stress_Level': stress_level,'Smoking': smoking,'Alcohol_Consumption': alcohol,'BMI': bmi,'Heart_Rate': heart_rate,'Physical_Activity_Hours': physical_activity,'Health_Issues': labels})
print("Dataset ready — shape:", df.shape)
df.head()

In [ ]:
# Блок 2 — Попередня обробка та інженерія ознак
X = df.drop(['ID','Health_Issues'], axis=1)
y = df['Health_Issues']
X['Coffee_Sleep_Interaction'] = X['Coffee_Intake'] * X['Sleep_Hours']
X['Age_Binned'] = pd.cut(X['Age'], bins=[17,30,50,80], labels=['Young','Middle','Old'])
cat_cols = ['Gender','Country','Occupation','Sleep_Quality','Stress_Level','Smoking','Alcohol_Consumption','Age_Binned']
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)
num_cols = ['Age','Coffee_Intake','Caffeine_mg','Sleep_Hours','BMI','Heart_Rate','Physical_Activity_Hours','Coffee_Sleep_Interaction']
scaler = StandardScaler()
X_encoded[num_cols] = scaler.fit_transform(X_encoded[num_cols])
print("Preprocessing finished. Feature count:", X_encoded.shape[1])
X_encoded.head()

In [ ]:
# Блок 3 — Розбиття на train/val/test (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(X_encoded, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)
print("Train/Val/Test shapes:", X_train.shape, X_val.shape, X_test.shape)
le = LabelEncoder(); y_train_enc = le.fit_transform(y_train); y_val_enc = le.transform(y_val); y_test_enc = le.transform(y_test)
class_names = le.classes_; print("Classes:", class_names)

In [ ]:
# Блок 4 — Реалізація MyDecisionTree (Gini, feature importance)
class MyDecisionTree:
    def __init__(self, max_depth=6, min_samples=5):
        self.max_depth = max_depth; self.min_samples = min_samples; self.feature_importance = None; self.tree = None; self.n_total = None
    def gini(self, y):
        if len(y) == 0: return 0.0
        _, counts = np.unique(y, return_counts=True); p = counts / counts.sum(); return 1.0 - np.sum(p**2)
    def best_split(self, X, y):
        best_gini = float('inf'); best_feat=None; best_thr=None; n,m = X.shape
        if n<=1: return None,None,None,None
        parent_gini = self.gini(y)
        for feature in range(m):
            vals = np.unique(X[:,feature])
            if len(vals)==1: continue
            sorted_vals = np.sort(vals); thresholds = (sorted_vals[:-1]+sorted_vals[1:])/2.0
            for thr in thresholds:
                left_idx = X[:,feature] <= thr; right_idx = ~left_idx
                if left_idx.sum() < self.min_samples or right_idx.sum() < self.min_samples: continue
                g_left = self.gini(y[left_idx]); g_right = self.gini(y[right_idx])
                g_weighted = (left_idx.sum()/n)*g_left + (right_idx.sum()/n)*g_right
                if g_weighted < best_gini:
                    best_gini = g_weighted; best_feat=feature; best_thr=thr
        return best_feat, best_thr, best_gini, parent_gini
    def build_tree(self, X, y, depth=0):
        node={}; num_samples = len(y); values,counts = np.unique(y,return_counts=True); majority_class = int(values[np.argmax(counts)])
        node['n_samples']=int(num_samples); node['class']=majority_class; node['gini']=float(self.gini(y))
        if depth>=self.max_depth or num_samples<self.min_samples or node['gini']==0.0: node['leaf']=True; return node
        feat,thr,best_gini,parent_gini = self.best_split(X,y)
        if feat is None: node['leaf']=True; return node
        node['leaf']=False; node['feature']=int(feat); node['threshold']=float(thr)
        n=num_samples; left_mask = X[:,feat] <= thr; right_mask = ~left_mask
        n_left = int(left_mask.sum()); n_right = int(right_mask.sum())
        g_left = self.gini(y[left_mask]); g_right = self.gini(y[right_mask])
        g_weighted = (n_left/n)*g_left + (n_right/n)*g_right
        importance_contrib = (n / self.n_total) * (parent_gini - g_weighted); self.feature_importance[feat] += importance_contrib
        node['left'] = self.build_tree(X[left_mask], y[left_mask], depth+1); node['right'] = self.build_tree(X[right_mask], y[right_mask], depth+1)
        return node
    def fit(self, X, y):
        X = np.array(X, dtype=float); y = np.array(y, dtype=int); self.n_total = len(y)
        self.feature_importance = np.zeros(X.shape[1], dtype=float); self.tree = self.build_tree(X,y,depth=0)
        s = np.sum(self.feature_importance); 
        if s>0: self.feature_importance = self.feature_importance / s
        return self
    def predict_one(self, x, node):
        if node.get('leaf',False): return int(node['class'])
        if x[node['feature']] <= node['threshold']: return self.predict_one(x, node['left'])
        else: return self.predict_one(x, node['right'])
    def predict(self, X):
        X = np.array(X, dtype=float); return np.array([self.predict_one(x,self.tree) for x in X], dtype=int)
    def count_leaves(self, node):
        if node.get('leaf',False): return 1
        return self.count_leaves(node['left']) + self.count_leaves(node['right'])
    def calculate_error(self, X_val, y_val, node):
        if len(y_val)==0: return 0.0
        preds = np.full(len(y_val), node['class'], dtype=int); return np.sum(preds != y_val) / len(y_val)
    def prune_tree(self, node, alpha, X_val, y_val):
        if node.get('leaf',False): return self.calculate_error(X_val, y_val, node)
        feat = node['feature']; thr = node['threshold']
        left_mask = X_val[:, feat] <= thr; right_mask = ~left_mask
        left_error = self.prune_tree(node['left'], alpha, X_val[left_mask], y_val[left_mask])
        right_error = self.prune_tree(node['right'], alpha, X_val[right_mask], y_val[right_mask])
        total_val = len(y_val)
        if total_val>0:
            def predict_subtree(x_row, node_local):
                if node_local.get('leaf',False): return node_local['class']
                if x_row[node_local['feature']] <= node_local['threshold']: return predict_subtree(x_row, node_local['left'])
                else: return predict_subtree(x_row, node_local['right'])
            preds = np.array([predict_subtree(x, node) for x in X_val])
            subtree_mis = np.sum(preds != y_val) / total_val
        else: subtree_mis = 0.0
        leaf_error = self.calculate_error(X_val, y_val, {'leaf':True, 'class': node['class']}); leaves = self.count_leaves(node)
        if (leaf_error + alpha) <= (subtree_mis + alpha * (leaves - 1)):
            node['leaf'] = True; node.pop('left', None); node.pop('right', None); node.pop('feature', None); node.pop('threshold', None)
            return leaf_error
        else:
            return subtree_mis

In [ ]:
# Блок 5 — Навчання MyDecisionTree та оцінка
tree = MyDecisionTree(max_depth=8, min_samples=10); tree.fit(X_train.values, y_train_enc)
pred_train = tree.predict(X_train.values); pred_val = tree.predict(X_val.values); pred_test = tree.predict(X_test.values)
acc_train = accuracy_score(y_train_enc, pred_train); acc_val = accuracy_score(y_val_enc, pred_val); acc_test = accuracy_score(y_test_enc, pred_test)
f1_test = f1_score(y_test_enc, pred_test, average='macro'); mcc_test = matthews_corrcoef(y_test_enc, pred_test); cm_test = confusion_matrix(y_test_enc, pred_test)
print("MyDecisionTree — before pruning: Train acc {:.3f}, Val acc {:.3f}, Test acc {:.3f}, Test F1 {:.3f}, MCC {:.3f}".format(acc_train, acc_val, acc_test, f1_test, mcc_test))
print("Confusion matrix (test):\n", cm_test)

In [ ]:
# Блок 6 — Прунінг: пошук alpha за валідацією та застосування
def prune_via_validation(tree_obj, X_val, y_val, alphas=[0.0,0.0005,0.001,0.002,0.005,0.01,0.02]):
    best_alpha=None; best_err=1.0; best_tree=None
    for a in alphas:
        tree_copy = copy.deepcopy(tree_obj.tree); old_tree = tree_obj.tree; tree_obj.tree = tree_copy
        tree_obj.prune_tree(tree_obj.tree, a, np.array(X_val), np.array(y_val))
        preds_val = np.array([tree_obj.predict_one(x, tree_obj.tree) for x in np.array(X_val)])
        val_err = np.sum(preds_val != np.array(y_val)) / len(y_val) if len(y_val)>0 else 0.0
        if val_err < best_err: best_err = val_err; best_alpha=a; best_tree = copy.deepcopy(tree_obj.tree)
        tree_obj.tree = old_tree
    return best_alpha, best_err, best_tree
best_alpha, best_err, best_tree = prune_via_validation(tree, X_val.values, y_val_enc)
print("Best alpha:", best_alpha, "validation error:", best_err)
if best_tree is not None:
    tree.tree = best_tree
    pred_test_pruned = tree.predict(X_test.values)
    acc_test_pruned = accuracy_score(y_test_enc, pred_test_pruned); f1_test_pruned = f1_score(y_test_enc, pred_test_pruned, average='macro'); mcc_test_pruned = matthews_corrcoef(y_test_enc, pred_test_pruned)
    cm_test_pruned = confusion_matrix(y_test_enc, pred_test_pruned)
    print("After pruning: Test acc {:.3f}, F1 {:.3f}, MCC {:.3f}".format(acc_test_pruned, f1_test_pruned, mcc_test_pruned))
else:
    pred_test_pruned = pred_test; cm_test_pruned = cm_test; acc_test_pruned = acc_test; f1_test_pruned = f1_test; mcc_test_pruned = mcc_test
    print("No pruning improvement found.")

In [ ]:
# Блок 7 — Порівняння з sklearn DecisionTree та RandomForest
dt_sk = DecisionTreeClassifier(criterion='gini', max_depth=8, min_samples_leaf=10, random_state=42); dt_sk.fit(X_train, y_train_enc); pred_dt_sk = dt_sk.predict(X_test)
rf = RandomForestClassifier(n_estimators=100, random_state=42); rf.fit(X_train, y_train_enc); pred_rf = rf.predict(X_test)
def compute_metrics(y_true, y_pred, name): return {'model':name, 'accuracy':accuracy_score(y_true,y_pred), 'f1_macro':f1_score(y_true,y_pred,average='macro'), 'mcc':matthews_corrcoef(y_true,y_pred)}
metrics = [compute_metrics(y_test_enc, pred_test, 'MyDecisionTree (unpruned)'), compute_metrics(y_test_enc, pred_test_pruned, 'MyDecisionTree (pruned)'), compute_metrics(y_test_enc, pred_dt_sk, 'sklearn DecisionTree'), compute_metrics(y_test_enc, pred_rf, 'RandomForest')]
metrics_df = pd.DataFrame(metrics); print(metrics_df.to_string(index=False))

In [ ]:
# Блок 8 — Візуалізації: confusion matrices та важливості ознак
import matplotlib.pyplot as plt; plt.rcParams.update({'figure.max_open_warning': 0})
fig, axs = plt.subplots(1,3, figsize=(15,4))
axs[0].imshow(confusion_matrix(y_test_enc, pred_test), interpolation='nearest'); axs[0].set_title('MyTree (before prune)')
axs[1].imshow(confusion_matrix(y_test_enc, pred_test_pruned), interpolation='nearest'); axs[1].set_title('MyTree (after prune)')
axs[2].imshow(confusion_matrix(y_test_enc, pred_dt_sk), interpolation='nearest'); axs[2].set_title('sklearn DecisionTree')
for ax in axs:
    ax.set_xticks(range(len(class_names))); ax.set_yticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45); ax.set_yticklabels(class_names)
    ax.set_xlabel('Predicted'); ax.set_ylabel('True')
plt.tight_layout(); plt.show()
feat_names = X_encoded.columns.tolist(); my_imp = tree.feature_importance; rf_imp = rf.feature_importances_
top_idx_my = np.argsort(my_imp)[-15:][::-1]
plt.figure(figsize=(12,4)); plt.barh([feat_names[i] for i in top_idx_my[::-1]], my_imp[top_idx_my[::-1]]); plt.title('Top features — MyDecisionTree importance (normalized)'); plt.xlabel('Importance'); plt.tight_layout(); plt.show()
top_idx_rf = np.argsort(rf_imp)[-15:][::-1]; plt.figure(figsize=(12,4)); plt.barh([feat_names[i] for i in top_idx_rf[::-1]], rf_imp[top_idx_rf[::-1]]); plt.title('Top features — RandomForest importance'); plt.xlabel('Importance'); plt.tight_layout(); plt.show()
perm = permutation_importance(rf, X_test, y_test_enc, n_repeats=5, random_state=42); perm_imp = perm.importances_mean; top_perm_idx = np.argsort(perm_imp)[-15:][::-1]
plt.figure(figsize=(12,4)); plt.barh([feat_names[i] for i in top_perm_idx[::-1]], perm_imp[top_perm_idx[::-1]]); plt.title('Top features — permutation importance (RandomForest)'); plt.xlabel('Importance'); plt.tight_layout(); plt.show()

In [ ]:
# Блок 9 — Feature selection за importance (top-10 від MyDecisionTree) та повторне навчання
top10 = [feat_names[i] for i in np.argsort(my_imp)[-10:]]; print("Top-10 features by MyDecisionTree:", top10)
X_train_sel = X_train[top10]; X_test_sel = X_test[top10]
dt_sk2 = DecisionTreeClassifier(criterion='gini', max_depth=8, min_samples_leaf=10, random_state=42); dt_sk2.fit(X_train_sel, y_train_enc); pred_dt2 = dt_sk2.predict(X_test_sel)
rf2 = RandomForestClassifier(n_estimators=100, random_state=42); rf2.fit(X_train_sel, y_train_enc); pred_rf2 = rf2.predict(X_test_sel)
metrics2 = [compute_metrics(y_test_enc, pred_dt2, 'sklearn DecisionTree (top10)'), compute_metrics(y_test_enc, pred_rf2, 'RandomForest (top10)')]; metrics2_df = pd.DataFrame(metrics2); print(metrics2_df.to_string(index=False))

In [ ]:
# Блок 10 — Висновки (шаблон)
print('--- ВИСНОВКИ ---') 
print('- Найкраща модель: (заповніть після запуску)') 
print('- Вплив прунінгу: (заповніть)') 
print('- Важливі ознаки: (перегляньте графіки)') 
print('- Рекомендації: (наприклад, додати cross-val, grid-search, tuning)')